In [15]:
import openai
from pathlib import Path

In [16]:
openai.api_key = open('openai_api_key.txt').read()

In [20]:
# task_type = 'use_model_output'
task_type = 'use_original_doc'
logic_type = 'N3'

In [21]:
prompts = [
    open(prompt_path).read()
    for prompt_path in sorted(Path(f'prompts/{task_type}/{logic_type}').glob(f'*.txt'))
]

In [26]:
all_outputs = {}
output_dir = Path('outputs')/task_type/logic_type

for test_idx,test_file in enumerate(sorted(Path('tests').glob('*.txt'))):
    test_input = open(test_file).read()
    model_output = test_input
    test_output_dir = output_dir / test_file.stem
    test_output_dir.mkdir(parents=True, exist_ok=True)
    test_outputs = []

    for task_idx, base_prompt in enumerate(prompts):
        if task_type == 'use_original_doc':
            prompt = '{}\n"""{}"""'.format(base_prompt, test_input)
        elif task_type == 'use_model_output':
            prompt = '{}\n"""{}"""'.format(base_prompt, model_output)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt},
            ]
        )
        model_output = response['choices'][0]['message']['content']
        test_outputs.append(model_output)

        print(f'Task {task_idx+1} Complete')
        
        # save output file
        open(test_output_dir / f'task{task_idx+1}.txt', 'w').write(model_output)
    print(f'Test {test_idx+1} complete')
    print('-----------------')
    all_outputs[test_file.stem] = test_outputs

Task 1 Complete


KeyboardInterrupt: 